In [11]:
%pip install matplotlib numpy 

Note: you may need to restart the kernel to use updated packages.


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

In [13]:
# MNIST 

img_rows, img_cols, channels = 28, 28, 1
img_shape = (channels, img_rows, img_cols)

z_dim = 100 # Dimension vom Rauschvektor
batch_size = 128
iterations = 20000



In [14]:
transform = transforms.Compose([
    transforms.ToTensor(), # [0,1]
    transforms.Normalize((0.5,), (0.5,)) # -> [-1, 1] 
])

mnist = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transform
)

dataloader = DataLoader(
    mnist,
    batch_size=batch_size,
    shuffle=True
)

In [ ]:
class Generator(nn.Module):
    def __init__(self, z_dim, img_shape):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(z_dim, 128), # Rauschen -> 128 
            nn.LeakyReLU(0.01), # Aktivierungsfunktion
            nn.Linear(128, np.prod(img_shape)),  # 128 -> Bildgröße
            nn.Tanh() # Output im Bereich [-1, 1]
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img
    
class Discriminator(nn.Module):
    def __init__(self, img_shape):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(), # Bild -> Vektor
            nn.Linear(np.prod(img_shape), 128), # Bildgröße -> 128 
            nn.LeakyReLU(0.01), # Aktivierungsfunktion
            nn.Linear(128, 1), # 128 -> Einzelwert
            nn.Sigmoid() # Output im Bereich [0, 1] weil binäre Klassifikation
        )
        # 28*28*128 + 128 + 2*128
    def forward(self, img):
        return self.model(img)

generator = Generator(z_dim, img_shape)
discriminator = Discriminator(img_shape)

BCE = nn.BCELoss() 

optimizer_G = optim.Adam(generator.parameters(), lr=1e-3) 
# Adam - moderne Variante des Gradientenabstiegs
optimizer_D = optim.Adam(discriminator.parameters(), lr=1e-3)

In [16]:
def sample_images(generator, z_dim, image_grid_rows=4, image_grid_columns=4):
    generator.eval()
    with torch.no_grad():
        z = torch.randn(image_grid_rows * image_grid_columns, z_dim)
        gen_imgs = generator(z).cpu()

    gen_imgs = (gen_imgs + 1) / 2

    fig, axs = plt.subplots(
        image_grid_rows,
        image_grid_columns,
        figsize=(4, 4),
        sharex=True,
        sharey=True
    )

    cnt = 0
    for i in range(image_grid_rows):
        for j in range(image_grid_columns):
            axs[i, j].imshow(gen_imgs[cnt, 0, :, :], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1
    plt.show()

In [17]:
# Training Loop
def train(iterations):
    iteration = 0
    while iteration < iterations:
        for real_imgs, _ in dataloader: # echte Bilder aus dem Datensatz
            iteration += 1
            batch_size_curr = real_imgs.size(0)

            # Labels für echte und gefälschte Bilder
            real = torch.ones(batch_size_curr, 1)
            fake = torch.zeros(batch_size_curr, 1)
            # L_D(G,D) = BCE(D(x^{real}),1) + BCE(D(G(z)),0)
            # Trainiere Discriminator
            discriminator.train()
            optimizer_D.zero_grad()

            # Part 1 - BCE(D(x^{real}),1)
            real_outputs = discriminator(real_imgs)
            d_loss_real = BCE(real_outputs, real)

            # Part 2 - BCE(D(G(z)),0)
            z = torch.randn(batch_size_curr, z_dim)
            fake_imgs = generator(z)
            fake_outputs = discriminator(fake_imgs.detach()) # detach - keine Gradienten für G
            d_loss_fake = BCE(fake_outputs, fake)

            d_loss = 0.5*(d_loss_real + d_loss_fake) # Gesamter Verlust für D = L_D(G,D) , gemittelt
            d_loss.backward()
            optimizer_D.step()

            # Trainiere Generator
            generator.train()
            optimizer_G.zero_grad()

            # L_G = BCE(D(G(z)),1)
            z = torch.randn(batch_size_curr, z_dim)
            fake_imgs = generator(z)
            fake_outputs = discriminator(fake_imgs) # keine detach - Gradienten für G

            g_loss = BCE(fake_outputs,real) 
            # Generator will, dass D die gefälschten Bilder als echt klassifiziert
            g_loss.backward()
            optimizer_G.step()

            if iteration % 1000 == 0:
                print(f"Iteration {iteration}, D Loss: {d_loss.item()}, G Loss: {g_loss.item()}")
                sample_images(generator, z_dim)
            
            if iteration >= iterations:
                break

In [ ]:
train(iterations)